## Mapping trajectory data to nodes of the graph


In [ ]:
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install Pysal
!pip install pysal --quiet

Open GeoJson files and import libraries

In [ ]:
from difflib import restore
import os
from traceback import print_tb
from geopandas.geoseries import GeoSeries
import matplotlib.pyplot as plt
import geopandas
import numpy as np
import json
from osgeo import gdal
import networkx as nx
from scipy.spatial import cKDTree
import pandas as pd
import itertools
from operator import itemgetter
import re
from libpysal import weights
beijing = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/beijing.geojson")
beijing.head()

gpd1 = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/trj200cars.geojson") 

Finding nearest node to the trajectory data and save trajectory as the correct structure

In [ ]:
def ckdnearest(gdfA, gdfB, gdfB_cols=['OBJECTID']):

    A = np.concatenate(
        [np.array(geom.coords) for geom in gdfA.geometry.to_list()])
    B = [np.array(geom.coords) for geom in gdfB.geometry.to_list()]
    B_ix = tuple(itertools.chain.from_iterable(
        [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))
    B = np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=1)
    idx = itemgetter(*idx)(B_ix)
    gdf = pd.concat(
        [gdfA, gdfB.loc[idx, gdfB_cols].reset_index(drop=True),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

result_map_trajectory_to_nodes= ckdnearest(gpd1, beijing)
print(str((result_map_trajectory_to_nodes['OBJECTID'])))

number_of_cars = 200
with open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/trj200cars.txt", 'a') as f:
    f.truncate(0)

    dfAsString = result_map_trajectory_to_nodes['OBJECTID'].to_string(header=False, index=False)
    dfAsString = re.sub("\s+", ",", dfAsString.strip())
    f.write(dfAsString)

       OBJECTID  OBJECTID
0             1     19059
1             2     19059
2             3     19059
3             4     19059
4             5     28932
...         ...       ...
35546     35547      3063
35547     35548      3063
35548     35549      3063
35549     35550      3063
35550     35551      3063

[35551 rows x 2 columns]


Removing id and repetitive items from trajectory

In [ ]:
######## removing id and repetitive items from trajectory
f=open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/trj200cars.txt","r")
lines=f.readlines()
trj_based_on_node_id=[]
trj_based_on_node_all=[]
i=1
print(str(len(lines[0].split(','))))
if (len(lines[0].split(','))%2)==0:
    kk = len(lines[0].split(','))
else:
    kk = (len(lines[0].split(','))-1)
for i in range(1, int(kk) ,2):
    trj_based_on_node_all.append(lines[0].split(',')[i])
    if (i > 1) & (lines[0].split(',')[i] != lines[0].split(',')[i-2]):
        trj_based_on_node_id.append(lines[0].split(',')[i])
    elif i==1:
        trj_based_on_node_id.append(lines[0].split(',')[i])
f.close()

f=open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/trj200cars.txt","w")
f.write(str(trj_based_on_node_id))

71102


139713

Organize paths to be able to detect current trajectory for each car

In [ ]:
gpd1['nearest_node'] = trj_based_on_node_all

array_trj_based_on_nodes_all_cars = []
for i in range(1, number_of_cars+1):
        one_car = gpd1[gpd1['id']==i]
        string_path = one_car['nearest_node'].to_string(header=False, index=False)
        string_path = re.sub("\s+", " ", string_path.strip())
        array_trj_based_on_nodes_all_cars.append(string_path)
       
  
with open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/trj200cars_classified.txt", 'a') as f:
    f.truncate(0)        
    f.write(str(array_trj_based_on_nodes_all_cars))   

 Find shortest path 

In [ ]:
queen_bei = weights.Queen.from_dataframe(beijing)
graph_bei = queen_bei.to_networkx()
def shortest_path(source, target):
    array_path = []
    #array_length =[]
    try:
         path = nx.all_shortest_paths(graph_bei, source=source, target=target)
         for path_test in nx.all_shortest_paths(graph_bei, source=source, target=target):
             array_path.append(path_test)
             #print(path_test)
         #length = nx.shortest_path_length(graph_bei,source=source, target=target)
         #array_length.append(length)
    except nx.NetworkXNoPath:
         path = "No Path"
         length = "No Pathlength"
         array_path.append(path)
         #array_length.append(length)
    return array_path

########## Source and Destination for each vehicle ################## 


array_five_routes_for_cars = []
for i in range(1, number_of_cars+1):
    one_car = gpd1[gpd1['id']==i]
    min = one_car['OBJECTID'].min()
    source_gpd = one_car[one_car['OBJECTID']==min]
    max = one_car['OBJECTID'].max()
    target_gpd = one_car[one_car['OBJECTID']==max]
    mm = 1
    while int(source_gpd['nearest_node'].mean()) == int(target_gpd['nearest_node'].mean()):
        #print("test null source_gpd = " + str(source_gpd['nearest_node'].isnull()))
        #print("test null target_gpd = " + str(target_gpd['nearest_node'].isnull()))
        if (max-mm == min) | (min == max):
            break
        target_gpd = one_car[one_car['OBJECTID']==max-mm]
        mm =mm+1
    #print("source_gpd for car "+str(i)+ "==" + str(source_gpd))
    #print("target_gpd for car "+str(i)+ "==" + str(target_gpd))
    #print("target for car "+str(i)+ "==" + str((source_gpd['nearest_node'].mean())))
    paths = shortest_path(int(source_gpd['nearest_node'].mean()), int(target_gpd['nearest_node'].mean()))
    #print("paths = " + str(paths))
    five_alt_for_one_car = []
    print("car=" + str(i) + "path=" + str(paths))
    for j in range(0, len(paths)):
        if j < 5:
            five_alt_for_one_car.append(paths[j])
    array_five_routes_for_cars.append(five_alt_for_one_car)
with open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/alternative_path.txt", 'a') as f:
    f.truncate(0)
    f.write(str(array_five_routes_for_cars).replace(', ',' ')) 

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/alternative_path.txt", 'a') as f:
    f.truncate(0)
    f.write(str(array_five_routes_for_cars).replace(', ',' ')) 

Cleaning alternative routes

In [ ]:
######### clean alternative routes to send to QC
### data structure should be as below:
### 0 19059-20075-33394-5755-4128-25791


f=open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/alternative_path.txt","r")
f = f.read()
f = f.replace('[[[','[').replace(']]]',']').replace(']] [[','],[').split(',')
print(f)
print('test=' +f[0])

array_all_cars = []
for i in range(0, len(f)):

    one_car = f[i].replace('] [',',').replace('[','').replace(']','').split(',')
    print("first car = " +str(one_car))
    print("len = " +str(len(one_car)))
    
    if len(one_car) > 1:
      one_path = str(i) +" "+ str(one_car[0].replace(' ','-'))
      second_path = str(i) +" "+ str(one_car[1].replace(' ','-'))
      array_all_cars.append(one_path)
      array_all_cars.append(second_path)
    elif len(one_car) == 1:
      one_path = str(i) +" "+ str(one_car[0].replace(' ','-'))
      array_all_cars.append(one_path)
      array_all_cars.append(one_path)
    elif len(one_car) == 0:
      array_all_cars.append([])
      array_all_cars.append([])

print("all_acrs len = " + str(len(array_all_cars)))
print("all_acrs = " + str(array_all_cars))
  



In [ ]:
textfile = open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/alternative_path_cleaned.txt", "w")
for element in array_all_cars:
  textfile. write(element + "\n")

Cleaning main routes

In [ ]:
f=open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/trj200cars_classified.txt","r")
f = f.read()
f = f.replace('\', \'',',').replace('\']','').replace('[\'','').replace(' ','-').split(',')
print(f)

['19059-19059-19059-19059-28932-19059-19059-19059-19059-28932-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-44090-19059-44090-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-19059-14099-5776-34321-10469-45036-10471-9735-10324-515-10309-578-11107-11107-11107-11107-9133-35138-35135-34586-9740-28932-19059-28932-19059-19059-19059-19059', '740-13331-179-11081-797-4108-16670-10147-7996-34326-8003-8003-756-756-13455-7498-7199-7108-570-15324-16047-4097-5615-13474-5909-7278-49495-5637-10323-10328-13101-48863-48866-15785-15900-9183-1200-499-499-578-11079-11081-179-863-863-11081-10499-15979-16047-85-9558-11120-11120-12842-12842-12842-12842-12842-12842-12842-12842-12842-12846-29770-10152-9592-14632-6639-2669-8463-778-6196-756-36-7861-7344-4591-7274-981-1048-5637-16038-10314-10314-577-24-6156-6156-10503-4102-692-9616-10163-17156-8459-846

In [ ]:
textfile = open("/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/1_Mapping trj data/classified_cleaned.txt", "w")
for element in f:
  print(element)
  textfile. write(element + "\n")